In [1]:
#!/usr/bin/python3
# coding: utf-8
# sapporo

In [8]:
import codecs
from datetime import datetime as dt
import datetime
import sys
from jma_csvdl import save_jma_data, parse_jma_csv
import json
import numpy as np
import os
import pandas as pd
import plotly
import plotly.express as px
import plotly.tools as tls
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline as offline
from plotly.subplots import make_subplots
import sys
from urllib.request import urlretrieve
from cov19utils import create_basic_plot_figure, \
    show_and_clear, moving_average, \
    blank2zero, csv2array, \
    get_twitter, tweet_with_image, \
    get_gpr_predict, FONT_NAME, DT_OFFSET, \
    download_if_needed, json2nparr, code2int, age2int, \
    get_populations, get_os_idx_of_arr, dump_val_in_arr, \
    calc_last1w2w_dif, create_basic_scatter_figure, \
    show_and_save_plotly
from scipy.optimize import curve_fit

In [ ]:
if False:
    #filename="kanagawa.geojson"
    filename="aomori.geojson"
    f=codecs.open(filename, "rb", encoding='utf-8')
    j=json.load(f)
    f.close()
    f=codecs.open(filename.replace("geojson", "min.geojson"), "wb", encoding='utf-8')
    f.write(json.dumps(j, ensure_ascii=False, separators=(',', ':')))
    f.close()

In [ ]:
#import geopandas as gpd
#https://plotly.com/python/network-graphs/
#https://plotly.com/python/scattermapbox/
#https://labs.mapbox.com/maki-icons/
df=pd.read_csv("csv/maps.csv", encoding='shift-jis', header=0)
#df = px.data.carshare()
#print(df)
fig = px.scatter_mapbox(df,
    title='北海道 新型コロナ 発生マップ',
    lat="lat", lon="lon", color="patients", size='indoor',
    color_continuous_scale=plotly.colors.sequential.Bluered,
    hover_name="name", 
    hover_data=["state", "category"],
    labels={'lat':'緯度', 'lon':'経度', 'state':'自治体',
            'patients':'感染者数', 'indoor':'室内',
            'category':'形態', 'name':'名称', 'id':'ID'},
    size_max=12, zoom=7, height=500)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.update_layout(template='plotly_dark')
fig.show()

In [9]:
#df = pd.read_csv("covid19_data.csv", header=0, encoding='shift-jis')

In [10]:
cov19data = []
data_file = "covid19_data.csv"
# CSVデータを整形する
with codecs.open(data_file, encoding="shift-jis") as f:
    l = f.readline() # 先頭行をスキップ
    while l:
        l = f.readline().replace("\r\n", "")
        arr = l.split(',')
        if len(arr) == 23:
            tracked = blank2zero(arr[20])
            untracked = blank2zero(arr[21])
            unknown_rate = (untracked / max(1.0, (tracked + untracked))) * 100.0
            #print(tracked, untracked, unknown_rate)
            cov19data.append(
                [
                    dt(int(arr[1]), int(arr[2]), int(arr[3])), # date
                    blank2zero(arr[4]),  # tests
                    blank2zero(arr[6]),  # positive
                    blank2zero(arr[19]), # postive rate [%]
                    tracked, # tracked 濃厚接触
                    untracked,  # untracked 濃厚接触以外
                    unknown_rate # 経路不明率
                ]
            )

print("Total: {}".format(len(cov19data)))

Total: 284


In [11]:
pos_rate_np = np.array(cov19data)


In [12]:
pos_rate_np

array([[datetime.datetime(2020, 1, 28, 0, 0), 0, 1, ..., 0, 1, 100.0],
       [datetime.datetime(2020, 1, 29, 0, 0), 0, 0, ..., 0, 0, 0.0],
       [datetime.datetime(2020, 1, 30, 0, 0), 0, 0, ..., 0, 0, 0.0],
       ...,
       [datetime.datetime(2020, 11, 4, 0, 0), 1289, 75, ..., 26, 49,
        65.33333333333333],
       [datetime.datetime(2020, 11, 5, 0, 0), 1021, 119, ..., 41, 78,
        65.54621848739495],
       [datetime.datetime(2020, 11, 6, 0, 0), 1705, 115, ..., 55, 60,
        52.17391304347826]], dtype=object)